In [ ]:
from coffea.lookup_tools.lookup_base import lookup_base

class SoftDropWeight(lookup_base):
    def _evaluate(self, pt, eta):
        gpar = np.array([1.00626, -1.06161, 0.0799900, 1.20454])
        cpar = np.array([1.09302, -0.000150068, 3.44866e-07, -2.68100e-10, 8.67440e-14, -1.00114e-17])
        fpar = np.array([1.27212, -0.000571640, 8.37289e-07, -5.20433e-10, 1.45375e-13, -1.50389e-17])
        genw = gpar[0] + gpar[1]*np.power(pt*gpar[2], -gpar[3])
        ptpow = np.power.outer(pt, np.arange(cpar.size))
        cenweight = np.dot(ptpow, cpar)
        forweight = np.dot(ptpow, fpar)
        weight = np.where(np.abs(eta) < 1.3, cenweight, forweight)
        return genw*weight

_softdrop_weight = SoftDropWeight()

def corrected_msoftdrop(fatjets):
    sf = _softdrop_weight(fatjets.pt, fatjets.eta)
    sf = np.maximum(1e-5, sf)
    dazsle_msd = (fatjets.subjets * (1 - fatjets.subjets.rawFactor)).sum()
    return dazsle_msd.mass * sf

In [ ]:
def getParticles(genparticles,lowid=22,highid=25,flags=['fromHardProcess', 'isLastCopy']):
    """
    returns the particle objects that satisfy a low id, 
    high id condition and have certain flags
    """
    absid = abs(genparticles.pdgId)
    return genparticles[
        ((absid >= lowid) & (absid <= highid))
        & genparticles.hasFlags(flags)
    ]

def match_HWWlepqq(genparticles,candidatefj):
    """
    return the number of matched objects (hWW*),daughters, 
    and gen flavor (enuqq, munuqq, taunuqq) 
    """
    higgs = getParticles(genparticles,25)
    is_hWW = ak.all(abs(higgs.children.pdgId)==24,axis=2)

    higgs = higgs[is_hWW]
    higgs_wstar = higgs.children[ak.argmin(higgs.children.mass,axis=2,keepdims=True)]
    higgs_w = higgs.children[ak.argmax(higgs.children.mass,axis=2,keepdims=True)]
    
    prompt_electron = getParticles(genparticles,11,11,['isPrompt','isLastCopy'])
    prompt_muon = getParticles(genparticles,13,13,['isPrompt', 'isLastCopy'])
    prompt_tau = getParticles(genparticles,15,15,['isPrompt', 'isLastCopy'])
    prompt_q = getParticles(genparticles,0,5,['fromHardProcess', 'isLastCopy'])
    prompt_q = prompt_q[abs(prompt_q.distinctParent.pdgId) == 24]
    
    dr_fj_quarks = candidatefj.delta_r(prompt_q)
    dr_fj_electrons = candidatefj.delta_r(prompt_electron)
    dr_fj_muons = candidatefj.delta_r(prompt_muon)
    dr_fj_taus = candidatefj.delta_r(prompt_tau)
    dr_daughters = ak.concatenate([dr_fj_quarks,dr_fj_electrons,dr_fj_muons,dr_fj_taus],axis=1)
    hWWlepqq_nprongs = ak.sum(dr_daughters<0.8,axis=1)
    
    n_electrons = ak.sum(prompt_electron.pt>0,axis=1)
    n_muons = ak.sum(prompt_muon.pt>0,axis=1)
    n_taus = ak.sum(prompt_tau.pt>0,axis=1)
    n_quarks = ak.sum(prompt_q.pt>0,axis=1)

    # 4(elenuqq),6(munuqq),8(taunuqq)
    hWWlepqq_flavor = (n_quarks==2)*1 + (n_electrons==1)*3 + (n_muons==1)*5 + (n_taus==1)*7
    
    matchedH = candidatefj.nearest(higgs, axis=1, threshold=0.8)
    matchedW = candidatefj.nearest(higgs_w, axis=1, threshold=0.8)
    matchedWstar = candidatefj.nearest(higgs_wstar, axis=1, threshold=0.8) 

    # 1 (H only), 4(W), 6(W star), 9(H, W and Wstar)
    hWWlepqq_matched = (
        (ak.sum(matchedH.pt > 0, axis=1)==1) * 1 
        + (ak.sum(ak.flatten(matchedW.pt > 0, axis=2), axis=1)==1) * 3 
        + (ak.sum(ak.flatten(matchedWstar.pt > 0, axis=2), axis=1)==1) * 5
    )
    
    # leptons matched
    dr_leptons = ak.concatenate([dr_fj_electrons,dr_fj_muons], axis=1)
    matched_leptons = dr_leptons < 0.8
    
    leptons = ak.concatenate([prompt_electron, prompt_muon], axis=1)
    leptons = leptons[matched_leptons]
    
    # leptons coming from W or W*
    leptons_mass = ak.firsts(leptons.distinctParent.mass)
    higgs_w_mass = ak.firsts(ak.flatten(higgs_w.mass))[ak.firsts(leptons.pt > 0)]
    higgs_wstar_mass = ak.firsts(ak.flatten(higgs_wstar.mass))[ak.firsts(leptons.pt > 0)]

    iswlepton = leptons_mass == higgs_w_mass
    iswstarlepton = leptons_mass == higgs_wstar_mass
    
    return hWWlepqq_flavor,hWWlepqq_matched,hWWlepqq_nprongs,matchedH,higgs,iswlepton,iswstarlepton

In [ ]:
import numpy as np
import scipy
import awkward as ak
import hist as hist2
from coffea import processor
from coffea.nanoevents.methods import candidate, vector
from coffea.analysis_tools import Weights, PackedSelection


class HwwProcessor(processor.ProcessorABC):
    def __init__(self, year="2017", jet_arbitration='met', el_wp="wp80"):
        self._year = year
        self._jet_arbitration = jet_arbitration
        self._el_wp = el_wp
        
        self._triggers = {
            2016: {
                'e': [
                    "Ele50_CaloIdVT_GsfTrkIdT_PFJet165",
                    "Ele115_CaloIdVT_GsfTrkIdT",
                    "Ele15_IsoVVVL_PFHT600",
                ],
                'mu': [
                    "Mu50",
                    "Mu55",
                    "Mu15_IsoVVVL_PFHT600",
                ],
            },
            2017: {
                'e': [
                    "Ele50_CaloIdVT_GsfTrkIdT_PFJet165",
                    "Ele115_CaloIdVT_GsfTrkIdT",
                    "Ele15_IsoVVVL_PFHT600",
                ],
                'mu': [
                    "Mu50",
                    "Mu15_IsoVVVL_PFHT600",
                ],
            },
            2018: {
                'e': [
                    "Ele50_CaloIdVT_GsfTrkIdT_PFJet165",
                    "Ele115_CaloIdVT_GsfTrkIdT",
                    "Ele15_IsoVVVL_PFHT600",
                ],
                'mu': [
                    "Mu50",
                    "Mu15_IsoVVVL_PFHT600",
                ],
            }
        }
        self._triggers = self._triggers[int(self._year)]

        self.make_output = lambda: {
            'sumw': 0.,
            'signal_kin': hist2.Hist(
                hist2.axis.StrCategory([], name="region", growth=True),
                hist2.axis.IntCategory([0, 2, 4, 6, 8], name='genflavor', label='gen flavor'),
                hist2.axis.IntCategory([0, 1, 4, 6, 9], name='genHflavor', label='higgs matching'),
                hist2.axis.IntCategory([0, 1, 2, 3, 4], name='nprongs', label='Jet nprongs'),
                hist2.storage.Weight(),
            ),
            "jet_kin": hist2.Hist(
                hist2.axis.StrCategory([], name='region', growth=True),
                hist2.axis.Regular(30, 200, 1000, name='jetpt', label=r'Jet $p_T$ [GeV]'),
                hist2.axis.Regular(30, 10, 200, name="jetmsd", label="Jet $m_{sd}$ [GeV]"),
                hist2.axis.Regular(25, -20, 0, name="jetrho", label=r"Jet $\rho$"),
                hist2.axis.Regular(20, 0, 1, name="btag", label="Jet btag (opphem)"),
                hist2.storage.Weight(),
            ),
            "lep_kin": hist2.Hist(
                hist2.axis.StrCategory([], name="region", growth=True),
                hist2.axis.Regular(25, 0, 1, name="lepminiIso", label="lep miniIso"),
                hist2.axis.Regular(25, 0, 1, name="leprelIso", label="lep Rel Iso"),
                hist2.axis.Regular(40, 10, 800, name='lep_pt', label=r'lep $p_T$ [GeV]'),
                hist2.axis.Regular(30, 0, 5, name="deltaR_lepjet", label="$\Delta R(l, Jet)$"),
                hist2.storage.Weight(),
            ),
            "higgs_kin": hist2.Hist(
                hist2.axis.StrCategory([], name="region", growth=True),
                hist2.axis.Regular(50, 10, 1000, name='matchedHpt', label=r'matched H $p_T$ [GeV]'),
                hist2.axis.Variable(
                    [10,35,60,85,110,135,160,185,210,235,260,285,310,335,360,385,410,450,490,530,570,615,665,715,765,815,865,915,965],
                    name='genHpt', 
                    label=r'genH $p_T$ [GeV]',
                ),
                hist2.storage.Weight(),
            ),
            "met_kin": hist2.Hist(
                hist2.axis.StrCategory([], name="region", growth=True),
                hist2.axis.Regular(30, 0, 500, name="met", label=r"$p_T^{miss}$ [GeV]"),
                hist2.storage.Weight(),
            ),
        }
        
    def process(self, events):
        dataset = events.metadata['dataset']
        selection = PackedSelection()
        isRealData = not hasattr(events, "genWeight")
        nevents = len(events)
        weights = Weights(nevents, storeIndividual=True)
        
        output = self.make_output()
        if not isRealData:
            output['sumw'] = ak.sum(events.genWeight)
            weights.add("genweight", events.genWeight)
            
        # trigger
        for channel in ["e","mu"]:
            if isRealData:
                trigger = np.zeros(len(events), dtype='bool')
                for t in self._triggers[channel]:
                    if t in events.HLT.fields:
                        trigger = trigger | events.HLT[t]
                selection.add('trigger'+channel, trigger)
                del trigger
            else:
                selection.add('trigger'+channel, np.ones(nevents, dtype='bool'))

        # leptons
        goodmuon = (
            (events.Muon.pt > 25)
            & (abs(events.Muon.eta) < 2.4)
            & events.Muon.mediumId
        )
        nmuons = ak.sum(goodmuon, axis=1)
        lowptmuon = (
            (events.Muon.pt > 10)
            & (abs(events.Muon.eta) < 2.4)
            & events.Muon.looseId
        )
        nlowptmuons = ak.sum(lowptmuon, axis=1)
            
        if self._el_wp == "wp80":
            goodelectron = (
                (events.Electron.pt > 25)
                & (abs(events.Electron.eta) < 2.5)
                & (events.Electron.mvaFall17V2noIso_WP80)
            )
        elif self._el_wp == "wp90":
            goodelectron = (
                (events.Electron.pt > 25)
                & (abs(events.Electron.eta) < 2.5)
                & (events.Electron.mvaFall17V2noIso_WP90)
            )
        elif self._el_wp == "wpl":
            goodelectron = (
                (events.Electron.pt > 25)
                & (abs(events.Electron.eta) < 2.5)
                & (events.Electron.mvaFall17V2noIso_WPL)
            )
        elif not self.el_wp:
            goodelectron = (
                (events.Electron.pt > 25)
                & (abs(events.Electron.eta) < 2.5)
            )
        else:
            raise RuntimeError("Unknown working point")
                
        nelectrons = ak.sum(goodelectron, axis=1)
        lowptelectron = (
            (events.Electron.pt > 10)
            & (abs(events.Electron.eta) < 2.5)
            & (events.Electron.cutBased >= events.Electron.LOOSE)
        )
        nlowptelectrons = ak.sum(lowptelectron, axis=1)

        goodtau = (
            (events.Tau.pt > 20)
            & (abs(events.Tau.eta) < 2.3)
            & (events.Tau.idAntiEle >= 8)
            & (events.Tau.idAntiMu >= 1)
        )
        ntaus = ak.sum(goodtau, axis=1)
            
        selection.add('onemuon', (nmuons == 1) & (nlowptmuons <= 1) & (nelectrons == 0) & (nlowptelectrons == 0) & (ntaus == 0))
        selection.add('oneelectron', (nelectrons == 1) & (nlowptelectrons <= 1) & (nmuons == 0) & (nlowptmuons == 0) & (ntaus == 0))
            
        # concatenate leptons and select leading one
        goodleptons = ak.concatenate([events.Muon[goodmuon], events.Electron[goodelectron]], axis=1)
        candidatelep = ak.firsts(goodleptons[ak.argsort(goodleptons.pt)])
        candidatelep_p4 = ak.zip(
            {
                "pt": candidatelep.pt,
                "eta": candidatelep.eta,
                "phi": candidatelep.phi,
                "mass": candidatelep.mass,
                "charge": candidatelep.charge,
            },
            with_name="PtEtaPhiMCandidate",
            behavior=candidate.behavior,
        )
            
        # missing transverse energy
        met = events.MET

        # fatjets
        fatjets = events.FatJet
        fatjets["msdcorr"] = corrected_msoftdrop(fatjets)
        fatjets["qcdrho"] = 2 * np.log(fatjets.msdcorr / fatjets.pt)
        
        candidatefj = fatjets[
            (fatjets.pt > 200)
        ]
        dphi_met_fj = abs(candidatefj.delta_phi(met))
        dr_lep_fj = candidatefj.delta_r(candidatelep_p4)

        if self._jet_arbitration == 'pt':
            candidatefj = ak.firsts(candidatefj)
        elif self._jet_arbitration == 'met':
            candidatefj = ak.firsts(candidatefj[ak.argmin(dphi_met_fj,axis=1,keepdims=True)])
        elif self._jet_arbitration == 'lep':
            candidatefj = ak.firsts(candidatefj[ak.argmin(dr_lep_fj,axis=1,keepdims=True)])
        else:
            raise RuntimeError("Unknown candidate jet arbitration")
            
            
        # lepton isolation
        lep_miniIso = candidatelep.miniPFRelIso_all
        lep_relIso = candidatelep.pfRelIso03_all
        
        selection.add("mu_pt", lep_miniIso < 0.2)
        selection.add("el_pt", lep_relIso < 0.2)
                
        # leptons within fatjet
        lep_in_fj = candidatefj.delta_r(candidatelep_p4) < 0.8
        lep_in_fj = ak.fill_none(lep_in_fj, False)
        selection.add("lep_in_fj", lep_in_fj)
        
        # jets
        jets = events.Jet
        jets = jets[
            (jets.pt > 30) 
            & (abs(jets.eta) < 2.5) 
            & jets.isTight
        ][:,:4]
        
        dphi_jet_fj = abs(jets.delta_phi(candidatefj))
        dr_jet_fj = abs(jets.delta_r(candidatefj))
        
        # b-jets
        bjets_ophem = ak.max(jets[dphi_jet_fj > np.pi / 2].btagDeepFlavB, axis=1)
        selection.add("btag_ophem", bjets_ophem > 0)

        # match HWWlepqq 
        if "HWW" in dataset:
            hWWlepqq_flavor,hWWlepqq_matched,hWWlepqq_nprongs,matchedH,genH,iswlepton,iswstarlepton = match_HWWlepqq(events.GenPart,candidatefj)
            matchedH_pt = ak.firsts(matchedH.pt)
            genH_pt = ak.firsts(genH.pt)
        else:
            hWWlepqq_flavor = ak.zeros_like(candidatefj.pt) 
            hWWlepqq_matched = ak.zeros_like(candidatefj.pt)
            hWWlepqq_nprongs = ak.zeros_like(candidatefj.pt)
            matchedH = ak.zeros_like(candidatefj.pt)
            matchedH_pt = ak.zeros_like(candidatefj.pt)
            genH = ak.zeros_like(candidatefj.pt)
            genH_pt = ak.zeros_like(candidatefj.pt)
            iswlepton = ak.ones_like(candidatefj.pt, dtype=bool)
            iswstarlepton = ak.ones_like(candidatefj.pt, dtype=bool)
            
        selection.add("iswlepton", iswlepton)
        selection.add("iswstarlepton", iswstarlepton)
        
        regions = {
            "hadel": ["lep_in_fj", "triggere", "oneelectron"],
            "hadmu": ["lep_in_fj", "triggermu", "onemuon"],
            "iswlepton": ["lep_in_fj", "iswlepton"],
            "iswstarlepton": ["lep_in_fj", "iswstarlepton"],
            "hadel_pt": ["lep_in_fj", "triggere", "oneelectron", "el_pt"],
            "hadmu_pt": ["lep_in_fj", "triggermu", "onemuon", "mu_pt"],
            "noselection": []
        }

        # function to normalize arrays after a cut or selection
        def normalize(val, cut=None):
            if cut is None:
                ar = ak.to_numpy(ak.fill_none(val, np.nan))
                return ar
            else:
                ar = ak.to_numpy(ak.fill_none(val[cut], np.nan))
                return ar
        
        def fill(region):
            selections = regions[region]
            cut = selection.all(*selections)
            
            output['signal_kin'].fill(
                region=region,
                genflavor=normalize(hWWlepqq_flavor, cut),
                genHflavor=normalize(hWWlepqq_matched, cut),
                nprongs=normalize(hWWlepqq_nprongs, cut),
                weight = weights.weight()[cut],
            )
            output["jet_kin"].fill(
                region=region,
                jetpt=normalize(candidatefj.pt, cut),
                jetmsd=normalize(candidatefj.msdcorr, cut),
                jetrho=normalize(candidatefj.qcdrho, cut),
                btag=normalize(bjets_ophem, cut),
                weight=weights.weight()[cut],
            )
            output['lep_kin'].fill(
                region=region,
                lepminiIso=normalize(lep_miniIso, cut),
                leprelIso=normalize(lep_relIso, cut),
                lep_pt=normalize(candidatelep.pt, cut),
                deltaR_lepjet=normalize(candidatefj.delta_r(candidatelep_p4), cut),
                weight=weights.weight()[cut],
            )
            output['higgs_kin'].fill(
                region=region,
                matchedHpt=normalize(matchedH_pt, cut),
                genHpt=normalize(genH_pt, cut),
                weight=weights.weight()[cut],
            )
            output["met_kin"].fill(
                region=region,
                met=normalize(met.pt, cut),
                weight=weights.weight()[cut],
            )
            
        for region in regions:
                fill(region)

        return {dataset: output}
            
    def postprocess(self, accumulator):
        return accumulator

In [ ]:
from dask.distributed import Client

client = Client("tls://daniel-2eocampo-2ehenao-40cern-2ech.dask.coffea.casa:8786")
client

In [ ]:
fileset = {
    'HWW': ["root://xcache/" + file for file in np.loadtxt("data/hwwdata.txt", dtype=str)]
}

hwwout = processor.run_uproot_job(
    fileset,
    treename="Events",
    processor_instance=HwwProcessor(),
    executor=processor.dask_executor,
    executor_args={
        "schema": processor.NanoAODSchema,
        "client": client,
    },
)

In [ ]:
fileset = {
    "tt": 
    ["root://xcache/" + file for file in np.loadtxt("data/ttsemileptonicdata.txt", dtype=str)]
}

ttout = processor.run_uproot_job(
    fileset,
    treename="Events",
    processor_instance=HwwProcessor(),
    executor=processor.dask_executor,
    executor_args={
        "schema": processor.NanoAODSchema,
        "client": client,
    },
)

In [ ]:
fileset = {
    "qcd": 
    ["root://xcache/" + file for file in np.loadtxt("data/qcdht100to200.txt", dtype=str)]
}

qcdout = processor.run_uproot_job(
    fileset,
    treename="Events",
    processor_instance=HwwProcessor(),
    executor=processor.dask_executor,
    executor_args={
        "schema": processor.NanoAODSchema,
        "client": client,
    },
)

In [ ]:
fileset = {
    "qcd": 
    ["root://xcache/" + file for file in np.loadtxt("data/qcdht200to300.txt", dtype=str)] 
}

qcdout2 = processor.run_uproot_job(
    fileset,
    treename="Events",
    processor_instance=HwwProcessor(),
    executor=processor.dask_executor,
    executor_args={
        "schema": processor.NanoAODSchema,
        "client": client,
    },
)

In [ ]:
fileset = {
    "qcd": 
    ["root://xcache/" + file for file in np.loadtxt("data/qcdht300to500data.txt", dtype=str)] 
}

qcdout3 = processor.run_uproot_job(
    fileset,
    treename="Events",
    processor_instance=HwwProcessor(),
    executor=processor.dask_executor,
    executor_args={
        "schema": processor.NanoAODSchema,
        "client": client,
    },
)

In [ ]:
fileset = {
    "qcd": 
    ["root://xcache/" + file for file in np.loadtxt("data/qcdht500to700data.txt", dtype=str)] 
}

qcdout4 = processor.run_uproot_job(
    fileset,
    treename="Events",
    processor_instance=HwwProcessor(),
    executor=processor.dask_executor,
    executor_args={
        "schema": processor.NanoAODSchema,
        "client": client,
    },
)

In [ ]:
fileset = {
    "qcd": 
    ["root://xcache/" + file for file in np.loadtxt("data/qcdht700to1000data.txt", dtype=str)] 
}

qcdout5 = processor.run_uproot_job(
    fileset,
    treename="Events",
    processor_instance=HwwProcessor(),
    executor=processor.dask_executor,
    executor_args={
        "schema": processor.NanoAODSchema,
        "client": client,
    },
)

In [ ]:
fileset = {
    "qcd": 
    ["root://xcache/" + file for file in np.loadtxt("data/qcdht1000to1500data.txt", dtype=str)] 
}

qcdout6 = processor.run_uproot_job(
    fileset,
    treename="Events",
    processor_instance=HwwProcessor(),
    executor=processor.dask_executor,
    executor_args={
        "schema": processor.NanoAODSchema,
        "client": client,
    },
)

In [ ]:
fileset = {
    "qcd": 
    ["root://xcache/" + file for file in np.loadtxt("data/qcdht1500to2000data.txt", dtype=str)] 
}

qcdout7 = processor.run_uproot_job(
    fileset,
    treename="Events",
    processor_instance=HwwProcessor(),
    executor=processor.dask_executor,
    executor_args={
        "schema": processor.NanoAODSchema,
        "client": client,
    },
)

In [ ]:
fileset = {
    "qcd": 
    ["root://xcache/" + file for file in np.loadtxt("data/qcdht2000toInfdata.txt", dtype=str)] 
}

qcdout8 = processor.run_uproot_job(
    fileset,
    treename="Events",
    processor_instance=HwwProcessor(),
    executor=processor.dask_executor,
    executor_args={
        "schema": processor.NanoAODSchema,
        "client": client,
    },
)

In [ ]:
fileset = {
    "wjets": 
    ["root://xcache/" + file for file in np.loadtxt("data/wjets100200.txt", dtype=str)] 
}

wout1 = processor.run_uproot_job(
    fileset,
    treename="Events",
    processor_instance=HwwProcessor(),
    executor=processor.dask_executor,
    executor_args={
        "schema": processor.NanoAODSchema,
        "client": client,
    },
)

In [ ]:
fileset = {
    "wjets": 
    ["root://xcache/" + file for file in np.loadtxt("data/wjets200400.txt", dtype=str)] 
}

wout2 = processor.run_uproot_job(
    fileset,
    treename="Events",
    processor_instance=HwwProcessor(),
    executor=processor.dask_executor,
    executor_args={
        "schema": processor.NanoAODSchema,
        "client": client,
    },
)

In [ ]:
fileset = {
    "wjets": 
    ["root://xcache/" + file for file in np.loadtxt("data/wjets400600.txt", dtype=str)] 
}

wout3 = processor.run_uproot_job(
    fileset,
    treename="Events",
    processor_instance=HwwProcessor(),
    executor=processor.dask_executor,
    executor_args={
        "schema": processor.NanoAODSchema,
        "client": client,
    },
)

In [ ]:
fileset = {
    "wjets": 
    ["root://xcache/" + file for file in np.loadtxt("data/wjets600800.txt", dtype=str)] 
}

wout4 = processor.run_uproot_job(
    fileset,
    treename="Events",
    processor_instance=HwwProcessor(),
    executor=processor.dask_executor,
    executor_args={
        "schema": processor.NanoAODSchema,
        "client": client,
    },
)

In [ ]:
fileset = {
    "wjets": 
    ["root://xcache/" + file for file in np.loadtxt("data/wjets8001200.txt", dtype=str)] 
}

wout5 = processor.run_uproot_job(
    fileset,
    treename="Events",
    processor_instance=HwwProcessor(),
    executor=processor.dask_executor,
    executor_args={
        "schema": processor.NanoAODSchema,
        "client": client,
    },
)

In [ ]:
fileset = {
    "wjets": 
    ["root://xcache/" + file for file in np.loadtxt("data/wjets12002500.txt", dtype=str)] 
}

wout6 = processor.run_uproot_job(
    fileset,
    treename="Events",
    processor_instance=HwwProcessor(),
    executor=processor.dask_executor,
    executor_args={
        "schema": processor.NanoAODSchema,
        "client": client,
    },
)

In [ ]:
fileset = {
    "wjets": 
    ["root://xcache/" + file for file in np.loadtxt("data/wjets2500Inf.txt", dtype=str)] 
}

wout7 = processor.run_uproot_job(
    fileset,
    treename="Events",
    processor_instance=HwwProcessor(),
    executor=processor.dask_executor,
    executor_args={
        "schema": processor.NanoAODSchema,
        "client": client,
    },
)

In [ ]:
fileset = {
    "singleElectron": 
    ["root://xcache/" + file for file in np.loadtxt("data/singleElectronB.txt", dtype=str)] +
    ["root://xcache/" + file for file in np.loadtxt("data/singleElectronC.txt", dtype=str)] +
    ["root://xcache/" + file for file in np.loadtxt("data/singleElectronD.txt", dtype=str)] +
    ["root://xcache/" + file for file in np.loadtxt("data/singleElectronE.txt", dtype=str)] +
    ["root://xcache/" + file for file in np.loadtxt("data/singleElectronF.txt", dtype=str)]
}

singleElectron = processor.run_uproot_job(
    fileset,
    treename="Events",
    processor_instance=HwwProcessor(),
    executor=processor.dask_executor,
    executor_args={
        "schema": processor.NanoAODSchema,
        "client": client,
    },
)

In [ ]:
import copy
import json

# cross sections
fname = "xsecs.json"
with open(fname) as file:
    xsecs = json.load(file)

    
def scale(hist, name, lumi=41500):
    """
    scale histograms from a one-key hist dictionary
    
    lumi in pb⁻1
    """
    dataset = mapped_dataset_names[name]
    dataset_key = list(hist.items())[0][0]
    dataset_sumw = hist[dataset_key]["sumw"]
    
    try:
        xsec = eval(xsecs[dataset])
    except:
        xsec = xsecs[dataset]
    
    weight = (xsec * lumi) / dataset_sumw
    
    hist = copy.deepcopy(hist)
    for h in hist[dataset_key].values():
        if isinstance(h, hist2.Hist):
            h *= (xsec * lumi) / dataset_sumw
    return hist


def get_scaled_hists(hists):
    """
    scale histograms from a multi-key hist dictionary
    """
    scaled_hists = {
        process:[] for process in mapped_dataset_names
    }
    
    for name, h in histos.items():
        scaled_hists[name] = scale(h, name)
        
    return scaled_hists


def accumulate(hists):
    """
    accumulate hists for qcd and wjets backgrounds
    """
    output = {
        "qcd":[],
        "wjets":[]
    }
    
    for key, value in hists.items():
        if "qcd" in key:
            output["qcd"].append(value)
        elif "wjets" in key:
            output["wjets"].append(value)
        else:
            output[key] = value
            
    output["qcd"] = processor.accumulate(output["qcd"])
    output["wjets"] = processor.accumulate(output["wjets"])
    
    return output

In [ ]:
# outputs
hists = [
    hww, 
    tt,
    qcd100to200, 
    qcd200to300, 
    qcd300to500, 
    qcd500to700, 
    qcd700to1000, 
    qcd1000to1500, 
    qcd1500to2000, 
    qcd2000toInf,
    wjets100to200, 
    wjets200to400,
    wjets400to600, 
    wjets600to800,
    wjets800to1200, 
    wjets1200to2500,
    wjets2500toInf
]

# map output names to cross section keys
mapped_dataset_names = {
    "hww": "hww",
    "ttsemileptonic": "TTToSemiLeptonic",
    "qcd100to200": "QCD_HT100to200",
    "qcd200to300": "QCD_HT200to300",
    "qcd300to500": "QCD_HT300to500",
    "qcd500to700": "QCD_HT500to700",
    "qcd700to1000": "QCD_HT700to1000",
    "qcd1000to1500": "QCD_HT1000to1500",
    "qcd1500to2000": "QCD_HT1500to2000",
    "qcd2000toInf": "QCD_HT2000toInf",
    "wjets100to200": "WJetsToLNu_HT-100To200",
    "wjets200to400": "WJetsToLNu_HT-200To400",
    "wjets400to600": "WJetsToLNu_HT-400To600",
    "wjets600to800": "WJetsToLNu_HT-600To800",
    "wjets800to1200": "WJetsToLNu_HT-800To1200",
    "wjets1200to2500": "WJetsToLNu_HT-1200To2500",
    "wjets2500toInf": "WJetsToLNu_HT-2500ToInf",
}

histos = {
    process:hist for process, hist in zip(mapped_dataset_names, hists)
}

In [ ]:
# scale histos
H = get_scaled_hists(histos)

# accumulate backgrounds
H = accumulate(H)

hww = H["hww"]["HWW"]
qcd = H["qcd"]["qcd"]
w = H["wjets"]["wjets"]
tt = H["ttsemileptonic"]["tt"]